In [ ]:
import time
time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.localtime(105484586906))

'Sat, 03 Sep 5312 10:08:26 +0000'

##Practical Example of BPNN

credit: https://machinelearningmastery.com/implement-backpropagation-algorithm-scratch-python/

In [ ]:
from random import seed
from random import random

from math import exp
import numpy as np
import pandas as pd


In [ ]:
 # Initialize a network, and randomly assign the weights
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network
 
seed(10)
network = initialize_network(2, 1, 1)
for layer in network:
	print(layer)

[{'weights': [0.5714025946899135, 0.4288890546751146, 0.5780913011344704]}]
[{'weights': [0.20609823213950174, 0.81332125135732]}]


In [ ]:
#manual assignment of weights
network = [[{'weights': [0.5, 0.1, 1]}],
 [{'weights': [1, -1]}]]

network

[[{'weights': [0.5, 0.1, 1]}], [{'weights': [1, -1]}]]

In [ ]:
#Forward propagation


# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return np.round(1.0 / (1.0 + exp(-activation)),2)

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs) #linear combination
			neuron['output'] = transfer(activation) # activation function e.g. sigmoid
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs



In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])



In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

In [ ]:
#Apply this function to learn from data - you must provide the train variable which consist of your dataset

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
  for epoch in range(n_epoch):
    sum_error = 0
    for row in train:
      outputs = forward_propagate(network, row[:4])
      expected = row[4:5]
      sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
      backward_propagate_error(network, expected)
      update_weights(network, row, l_rate)
      # for layer in network:
        # print(layer)
    if epoch%5 == 0:
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    


In [ ]:
df = pd.read_csv('train.csv')
df[["class"]] = df[["class"]].apply(lambda col:pd.Categorical(col).codes)
dataset = df.to_numpy().tolist()
dataset

[[5.1, 3.5, 1.4, 0.2, 0.0],
 [4.9, 3.0, 1.4, 0.2, 0.0],
 [4.7, 3.2, 1.3, 0.2, 0.0],
 [4.6, 3.1, 1.5, 0.2, 0.0],
 [5.0, 3.6, 1.4, 0.2, 0.0],
 [5.4, 3.9, 1.7, 0.4, 0.0],
 [4.6, 3.4, 1.4, 0.3, 0.0],
 [5.0, 3.4, 1.5, 0.2, 0.0],
 [4.4, 2.9, 1.4, 0.2, 0.0],
 [5.2, 3.5, 1.5, 0.2, 0.0],
 [5.2, 3.4, 1.4, 0.2, 0.0],
 [4.7, 3.2, 1.6, 0.2, 0.0],
 [4.8, 3.1, 1.6, 0.2, 0.0],
 [5.4, 3.4, 1.5, 0.4, 0.0],
 [7.0, 3.2, 4.7, 1.4, 1.0],
 [6.4, 3.2, 4.5, 1.5, 1.0],
 [6.9, 3.1, 4.9, 1.5, 1.0],
 [5.5, 2.3, 4.0, 1.3, 1.0],
 [6.5, 2.8, 4.6, 1.5, 1.0],
 [5.7, 2.8, 4.5, 1.3, 1.0],
 [6.3, 3.3, 4.7, 1.6, 1.0],
 [4.9, 2.4, 3.3, 1.0, 1.0],
 [6.6, 2.9, 4.6, 1.3, 1.0],
 [5.2, 2.7, 3.9, 1.4, 1.0],
 [5.6, 2.7, 4.2, 1.3, 1.0],
 [5.7, 3.0, 4.2, 1.2, 1.0],
 [5.7, 2.9, 4.2, 1.3, 1.0],
 [6.2, 2.9, 4.3, 1.3, 1.0],
 [5.1, 2.5, 3.0, 1.1, 1.0]]

In [1]:
# Test training backprop algorithm
seed(1)

n_inputs = len(dataset[0]) - 1
n_hidden =  1
n_outputs = 1
network = initialize_network(n_inputs, n_hidden, n_outputs)

#use it if you want to determine initial weights:
# network = [[{'weights': [0.5, 0.1, 1]}],
#            [{'weights': [1, -1]}]]

epoch = 500
lr = 0.5
train_network(network, dataset, lr, epoch, n_outputs)
# for layer in network:
# 	print(layer)

NameError: ignored

In [ ]:
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs[0]

## Evaluation

In [ ]:
def Evaluation(dataset):
    actual = []
    preds = []
    for row in dataset:
        actual.append(row[-1])
        prediction = predict(network, row)
        preds.append(int(np.round(prediction)))
        print('Expected=%d, Got (actual)=%f, Got (prediction)=%d' % (row[-1], prediction, np.round(prediction)))

    result  = pd.DataFrame((actual,preds)).T
    result.columns  = ("actual", "preds")

    conf_matrix  =  pd.crosstab(result.actual, result.preds)
    conf_matrix

    tn  = conf_matrix.iloc[0,0]
    fp  = conf_matrix.iloc[0,1]
    fn  = conf_matrix.iloc[1,0]
    tp  = conf_matrix.iloc[1,1]

    accuracy =  (tp+tn)/(tp+tn+fp+fn) * 100
    recall =  tp/(tp+fn) * 100
    precision  = tp/(tp+fp) * 100
    f1Score = 2*(recall*precision)/(recall+precision)

    print ('accuracy =  %.3f' % accuracy)
    print ('recall =  %.3f'  % recall)
    print ('precision =  %.3f' % precision)
    print ('f1Score =  %.3f'  % f1Score)



In [ ]:
df = pd.read_csv('val.csv')
df[["class"]] = df[["class"]].apply(lambda col:pd.Categorical(col).codes)
val = df.to_numpy().tolist()
val

[[4.9, 3.1, 1.5, 0.1, 0.0],
 [5.4, 3.7, 1.5, 0.2, 0.0],
 [4.8, 3.4, 1.6, 0.2, 0.0],
 [4.8, 3.0, 1.4, 0.1, 0.0],
 [4.8, 3.4, 1.9, 0.2, 0.0],
 [5.0, 2.0, 3.5, 1.0, 1.0],
 [5.9, 3.0, 4.2, 1.5, 1.0],
 [6.0, 2.2, 4.0, 1.0, 1.0],
 [6.1, 2.9, 4.7, 1.4, 1.0],
 [5.6, 2.9, 3.6, 1.3, 1.0]]

In [ ]:
df = pd.read_csv('test.csv')
df[["class"]] = df[["class"]].apply(lambda col:pd.Categorical(col).codes)
test = df.to_numpy().tolist()
test

[[4.3, 3.0, 1.1, 0.1, 0.0],
 [5.8, 4.0, 1.2, 0.2, 0.0],
 [5.7, 4.4, 1.5, 0.4, 0.0],
 [5.4, 3.9, 1.3, 0.4, 0.0],
 [5.1, 3.5, 1.4, 0.3, 0.0],
 [6.7, 3.1, 4.4, 1.4, 1.0],
 [5.6, 3.0, 4.5, 1.5, 1.0],
 [5.8, 2.7, 4.1, 1.0, 1.0],
 [6.2, 2.2, 4.5, 1.5, 1.0],
 [5.6, 2.5, 3.9, 1.1, 1.0]]

In [ ]:
Evaluation(dataset)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.9

In [ ]:
Evaluation(test)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
accuracy =  100.000
recall =  100.000
precision =  100.000
f1Score =  100.000


In [ ]:
Evaluation(val)

Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.020000, Got (prediction)=0
Expected=0, Got (actual)=0.030000, Got (prediction)=0
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
Expected=1, Got (actual)=0.980000, Got (prediction)=1
accuracy =  100.000
recall =  100.000
precision =  100.000
f1Score =  100.000


1. Ubah online learning menjadi batch learning
2. Modifikasi train network sehingga bisa menggunakan val data (per 5 epoch)
3. Evaluation menggunakan test data (bandingkan dengan accuracy training, validation)
